In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress



In [2]:
#Filtering down columns
#Disaster Decloration Summaries - disasterNumber, state, declarationType, fyDeclared, incidentType,  
#Hazard Mitigation Assistance Projects - disasterNumber, projectType, numberOfProperties, numberOfFinalProperties, projectAmount, federalShareObligated, benefitCostRatio, netValueBenefits
#Hazard Mitigation Grant Program Disaster Summaries - disasterNumber and obligatedTotalAmount

In [53]:
#Disaster Decloration Summaries - disasterNumber, state, declarationType, fyDeclared, incidentType,  
disaster_sum = pd.read_csv('DisasterDeclarationsSummaries.csv')
disaster_sum_clean = disaster_sum[['disasterNumber', 'state', 'declarationType', 'fyDeclared', 'incidentType', "declarationTitle"]]
disaster_sum_clean = disaster_sum_clean.drop_duplicates(subset = ["disasterNumber"])
disaster_sum_summary = pd.pivot_table(disaster_sum_clean,
                              values = ["declarationType"],
                              index = ["incidentType"],
#                              columns = ["fyDeclared"],
                              aggfunc = {"declarationType": "count"})
disaster_sum_summary.sort_values(by = ["declarationType"], ascending = False).head(4)

,declarationType
incidentType,
Fire,1403
Severe Storm(s),982
Flood,836
Hurricane,374


In [4]:
#Hazard Mitigation Assistance Projects - disasterNumber, projectType, numberOfProperties, numberOfFinalProperties, projectAmount, federalShareObligated, benefitCostRatio, netValueBenefits
haz_project = pd.read_csv('HazardMitigationAssistanceProjects.csv')
haz_project_clean = haz_project[['disasterNumber', 'projectType', 'numberOfProperties', 'numberOfFinalProperties', 'projectAmount', 'federalShareObligated', 'benefitCostRatio', 'netValueBenefits', 'projectIdentifier']]
haz_project_clean.head()

,disasterNumber,projectType,numberOfProperties,numberOfFinalProperties,projectAmount,federalShareObligated,benefitCostRatio,netValueBenefits,projectIdentifier
0,1971.0,206.1: Safe Room (Tornado and Severe Wind Shel...,5,4,31477.0,20086.0,1.371,21420.0,DR-1971-0561-R
1,1971.0,206.1: Safe Room (Tornado and Severe Wind Shel...,4,4,19032.0,13650.0,1.371,21420.0,DR-1971-0428-R
2,1971.0,206.2: Safe Room (Tornado and Severe Wind Shel...,1,0,769334.0,577000.0,4.460,3181553.0,DR-1971-0430-R
3,1971.0,601.1: Generators,0,0,42578.0,31934.0,0.000,0.0,DR-1971-0373-F
4,1971.0,206.2: Safe Room (Tornado and Severe Wind Shel...,1,1,143899.0,107924.0,3.097,500916.0,DR-1971-0672-R


In [26]:
disaster_projects = pd.merge(haz_project_clean, disaster_sum_clean, on='disasterNumber')
disaster_projects
disaster_summary = disaster_projects.groupby(['state','incidentType', 'disasterNumber', 'projectIdentifier'], as_index=False).agg({
    "numberOfProperties": "sum",
    "numberOfFinalProperties": "sum",
    "projectAmount": "sum",
    "federalShareObligated": "sum",
    "benefitCostRatio": "mean",
    "netValueBenefits": "sum",
    "fyDeclared": "mean"})
disaster_summary
disaster_summary = pd.pivot_table(disaster_projects,
                                  values = ["numberOfProperties", "numberOfFinalProperties", "projectAmount", "federalShareObligated", "benefitCostRatio", "netValueBenefits", "fyDeclared"],
                                 index = ["state", "incidentType", "disasterNumber", "projectIdentifier"],
                                 aggfunc = {"numberOfProperties": "sum",
                                           "numberOfFinalProperties": "sum",
                                           "projectAmount": "sum",
                                           "federalShareObligated": "sum",
                                           "benefitCostRatio": "mean",
                                           "netValueBenefits": "sum",
                                           "fyDeclared": "mean"})
disaster_summary
#disaster_summary.head(40)

benefitCostRatio  \
state incidentType    disasterNumber projectIdentifier                     
AK    Coastal Storm   4244.0         DR-4244-0001-P                0.000   
                                     DR-4244-0002-P                0.000   
                                     DR-4244-0003-R                2.020   
                                     DR-4244-0004-F                0.000   
                                     DR-4244-0007-M                0.000   
...                                                                  ...   
WY    Severe Storm(s) 4007.0         DR-4007-0008-R                1.313   
                                     DR-4007-0009-R                1.810   
                      4306.0         DR-4306-0003-M                0.000   
                                     DR-4306-0004-R                6.222   
      Tornado         1599.0         DR-1599-0001-P                0.000   

                                                        federalShareObligated  \
state incidentType    disasterNumber projectIdentifier                          
AK    Coastal Storm   4244.0         DR-4244-0001-P                  16526.00   
                                     DR-4244-0002-P                  18750.00   
                                     DR-4244-0003-R                  56004.49   
                                     DR-4244-0004-F                  11498.00   
                                     DR-4244-0007-M                  28718.00   
...                                                                       ...   
WY    Severe Storm(s) 4007.0         DR-4007-0008-R                9728048.00   
                                     DR-4007-0009-R                1812368.00   
                      4306.0         DR-4306-0003-M                  17417.00   
                                     DR-4306-0004-R                  95953.50   
      Tornado         1599.0         DR-1599-0001-P                  17089.00   

                                                        fyDeclared  \
state incidentType    disasterNumber projectIdentifier               
AK    Coastal Storm   4244.0         DR-4244-0001-P           2016   
                                     DR-4244-0002-P           2016   
                                     DR-4244-0003-R           2016   
                                     DR-4244-0004-F           2016   
                                     DR-4244-0007-M           2016   
...                                                            ...   
WY    Severe Storm(s) 4007.0         DR-4007-0008-R           2011   
                                     DR-4007-0009-R           2011   
                      4306.0         DR-4306-0003-M           2017   
                                     DR-4306-0004-R           2017   
      Tornado         1599.0         DR-1599-0001-P           2005   

                                                        netValueBenefits  \
state incidentType    disasterNumber projectIdentifier                     
AK    Coastal Storm   4244.0         DR-4244-0001-P                  0.0   
                                     DR-4244-0002-P                  0.0   
                                     DR-4244-0003-R             170513.0   
                                     DR-4244-0004-F                  0.0   
                                     DR-4244-0007-M                  0.0   
...                                                                  ...   
WY    Severe Storm(s) 4007.0         DR-4007-0008-R           25577280.0   
                                     DR-4007-0009-R            4559312.0   
                      4306.0         DR-4306-0003-M                  0.0   
                                     DR-4306-0004-R            2115692.0   
      Tornado         1599.0         DR-1599-0001-P                  0.0   

                                                        numberOfFinalProperties  \
state incidentType    disaster

In [24]:
len(disaster_summary)

22179

In [10]:
#Hazard Mitigation Grant Program Disaster Summaries
hazard_sum = pd.read_csv('HazardMitigationGrantProgramDisasterSummaries.csv')
hazard_sum_clean = hazard_sum[['disasterNumber', 'obligatedTotalAmount']]
hazard_sum_clean.head()

,disasterNumber,obligatedTotalAmount
0,1071,2834575.0
1,1337,782081.0
2,1326,333150.0
3,1506,1791043.0
4,1211,413384.0
